In [1]:
# Libraries for building network
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Libraries for dataset
import torchvision
import torchvision.transforms as transforms

# Miscellaneous Libraries
import time

In [16]:
class tutorial_model(nn.Module):
    def __init__(self):
        """ Initialize all layers of model """
        # CNN made from Learned Concepts from the Combination of ACM and Tutorial Provided
        super(tutorial_model, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=10, out_channels=20, kernel_size=5)
        
        self.conv2_dropout = nn.Dropout()                                                 
        
        self.fc1 = nn.Linear(in_features=320, out_features=200)
        self.fc2 = nn.Linear(in_features=200, out_features=10)
        # self.fc3 = nn.Linear(in_features=80, out_features=10)
        
    def forward(self, x):
        """ Chain all layers together """
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))          
        
        # Collapse 3D tensor to 1D
        x = x.view(-1, 320)                            
        
        x = self.fc1(x)
        y = self.fc2(x)
        # y = self.fc3(x)                                
        return y

In [17]:
# Number of epochs for training
num_epochs = 5           

# Batch Size for training/testing
batch_size = 64       

# Learning Rate for optimizer
learning_rate = 0.01     

# Momentum for optimizer
momentum = 0.5           

# Dimensions of MNIST
dim = 28

In [18]:
# Transformation for training data
transform_train = torchvision.transforms.Compose([
    transforms.ToTensor(),                            # Convert grayscale image to pytorch tensor
    transforms.Normalize((0.5,), (0.5,)),             # Normalize grayscale data
])

# Transformation for training data
transform_test = transforms.Compose([
    transforms.ToTensor(),                            # Convert grayscale image to pytorch tensor
    transforms.Normalize((0.5,), (0.5,)),             # Normalize grayscale data
])

In [19]:
# Download training data
trainset = torchvision.datasets.MNIST(root='./files', train=True, download=True, 
                                      transform=transform_train)

# Initialize dataloader for training data
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, 
                                           num_workers=8)

# Download testing Data
testset = torchvision.datasets.MNIST(root='./files', train=False, download=False, 
                                     transform=transform_test)

# Initialize dataloader for testing data
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, 
                                          num_workers=8)

In [20]:
# Initialize previously defined model
model = tutorial_model()                                               

# Definie loss function (Cross Entropy Loss)
criterion = nn.CrossEntropyLoss()                                      

# Initialize Optimizer (ADAM)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)     

# Set model to training (updating weights)
model.train();


In [21]:
# Store time to calculate train time
start_time = time.time()

# Store loss and accuracy data
loss = []
accuracy = []

# Train the model
# Loop for number of epochs
for epoch in range(num_epochs):
    # Loop through data in batch sized increments
    for batch_idx, (X_train_batch, Y_train_batch) in enumerate(train_loader):
        # If trained on all data in epoch, move onto next epoch
        if(Y_train_batch.shape[0]<batch_size):
            continue

        # Forward pass through network
        output = model(X_train_batch)                           
        # Calculate loss of predictions
        curr_loss = criterion(output, Y_train_batch)            
        # Store loss
        loss.append(curr_loss.item())                           

        
        # Clear last calculation
        optimizer.zero_grad()                                   
        # Calculate gradient based on loss
        curr_loss.backward()                                    
        # Update model weights
        optimizer.step()                                        

        # Extract model predictions
        _, predicted = torch.max(output.data, 1) 
        # Calculate number of correct predictions
        correct = (predicted == Y_train_batch).sum().item()     
        # Calculate/store accuracy
        accuracy.append(correct/Y_train_batch.size(0))          
        
        # Intermitently print statistics
        if batch_idx % 100 == 0:
            print('Epoch: ' + str(epoch+1) + '/' + str(num_epochs) + ', Step: ' 
                  + str(batch_idx+1) + '/' + str(len(train_loader)) + ', Loss: ' 
                  + str(curr_loss.item()) + ', Accuracy: ' 
                  + str(correct/Y_train_batch.size(0)*100) + '%')

# Store time to calculate train time
end_time = time.time()

# Print train time
print('Run Time: ' + str(end_time - start_time))

Epoch: 1/5, Step: 1/938, Loss: 2.2982754707336426, Accuracy: 14.0625%
Epoch: 1/5, Step: 101/938, Loss: 1.6123638153076172, Accuracy: 76.5625%
Epoch: 1/5, Step: 201/938, Loss: 0.3458239734172821, Accuracy: 87.5%
Epoch: 1/5, Step: 301/938, Loss: 0.21150200068950653, Accuracy: 93.75%
Epoch: 1/5, Step: 401/938, Loss: 0.3421831727027893, Accuracy: 84.375%
Epoch: 1/5, Step: 501/938, Loss: 0.1469740867614746, Accuracy: 98.4375%
Epoch: 1/5, Step: 601/938, Loss: 0.30795836448669434, Accuracy: 92.1875%
Epoch: 1/5, Step: 701/938, Loss: 0.20628461241722107, Accuracy: 95.3125%
Epoch: 1/5, Step: 801/938, Loss: 0.22404170036315918, Accuracy: 95.3125%
Epoch: 1/5, Step: 901/938, Loss: 0.2112247496843338, Accuracy: 96.875%
Epoch: 2/5, Step: 1/938, Loss: 0.14998292922973633, Accuracy: 95.3125%
Epoch: 2/5, Step: 101/938, Loss: 0.42787909507751465, Accuracy: 90.625%
Epoch: 2/5, Step: 201/938, Loss: 0.247516930103302, Accuracy: 93.75%
Epoch: 2/5, Step: 301/938, Loss: 0.13878120481967926, Accuracy: 96.875%
E

In [23]:
# Test the model
# Set model to testing (constant weights)
model.eval()

with torch.no_grad():
    # Store number of correct/total samples in test data
    correct = 0
    total = 0
    
    # Loop through test data
    for X_test_batch, Y_test_batch in test_loader:
        # Forward pass through network
        output = model(X_test_batch)  
        
        # Extract prediction
        _, predicted = torch.max(output.data, 1)    
        
        # Update total number of sample
        total += Y_test_batch.size(0)  
        
        # Update number of correct predictions
        correct += (predicted == Y_test_batch).sum().item()     

print('Test Accuracy: ' + str((correct/total) * 100) + '%')

Test Accuracy: 98.56%
